# Farmer Problem - Planting crops - Handle Uncertainity in Yield 


In [106]:
from pulp import *

In [107]:
# To show nice looking table of solution
from IPython.display import HTML, display

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4><td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

# Business Problem

### Total acerage = 500

|Field|Wheat|Corn|Beans|
|-----|-------|------|----------|
|Yield T/acre|2.5|3|20|
|Planting cost $/acre|150|230|260|
|Selling price|170|150|36 (<6000) 10 (>6000)|
|Purchase price|238|210|NA|
|Farm Needs|200|240|NA|


## Yields vary based on weather

Prob|Yield|Wheat|Corn|Beans|
----|-----|-------|------|----------|
0.333|Bad |2|2.4|16|
0.333|Average|2.5|3|20|
0.333|Good|3|3.6|24|

## Start defining data needed for problem

In [108]:
items = ["Wheat", "Corn","Beans"]
purchasable_items = ["Wheat", "Corn"]
total_acres = 500

### Define yields - we do not have to pick one yield - we will consider them all simultaneously

In [109]:
yields = {"BAD": {"Wheat":2,"Corn":2.4,"Beans":16}, 
          "AVERAGE" : {"Wheat":2.5,"Corn":3,"Beans":20},
          "GOOD": {"Wheat":3,"Corn":3.6,"Beans":24}}

### Define Planting Costs and Market purchase prices

In [110]:
planting_costs = {"Wheat":150,"Corn":230,"Beans":260}
purchase_prices = {"Wheat":238,"Corn":210,"Beans":999}

### Define consumption rules

In [111]:
consumption_feed={"Wheat":200,"Corn":240,"Beans":0}          

### Define selling prices - note the price difference for beans excess of 6000

In [112]:
selling_prices = {"Wheat":170,"Corn":150,"Beans":36}
selling_price_excess_beans = 10 

## We have to provide this to our model unlike our mean value problem where we picked only one scenario

In [113]:
scenario_p = {"BAD":0.333333333, "AVERAGE":0.333333333, "GOOD":0.333333333}

## Numbers that will help us understand tradeoffs

In [114]:
M = LpProblem("Farmer", LpMaximize)

# Define Variables

## First Stage - same as simple Mean value Linear Program

In [115]:
#Same variables as mean value problem
var_acres_planted = LpVariable.dicts("plant_acres", items, lowBound=0, cat='Continuous')

## Second stage will have variables for each scenario

In [116]:
var_tons_purchased_by_scenario = {}
var_tons_sold_by_scenario = {}
var_excess_beans_sold_by_scenario = {}

scenario_keys = list(scenario_p.keys())

for scenario_name in scenario_keys:
    var_tons_purchased_by_scenario[scenario_name] = LpVariable.dicts(scenario_name+"_purchase_tons", purchasable_items,lowBound=0,cat='Continuous') #no purchaefor beans
    var_tons_sold_by_scenario[scenario_name] = LpVariable.dicts(scenario_name+"_sell_tons", items,lowBound=0,cat='Continuous')
    var_excess_beans_sold_by_scenario[scenario_name] = LpVariable(scenario_name+'_sell_tons_extra_beans', lowBound=0,cat='Continuous')


# Define Objective

## Planting costs are summation over each item
## Selling revenue and buying costs are summation over each item and each scenario

In [117]:
M += lpSum(    [ -1*planting_costs[item] * var_acres_planted[item] for item in items ]   
           + [ -1* scenario_p[s] * purchase_prices[item] * var_tons_purchased_by_scenario[s][item] for item in purchasable_items for s in scenario_keys]
+ [ -1* scenario_p[s] * -1*selling_prices[item] * var_tons_sold_by_scenario[s][item] for item in items for s in scenario_keys]     
          + [scenario_p[s] * selling_price_excess_beans * var_excess_beans_sold_by_scenario[s] for s in scenario_keys] )


# Constraints

In [118]:
M += lpSum([var_acres_planted[i] for i in items]) <= total_acres

for s in scenario_keys:
#Corn, Wheat
    for item in purchasable_items:
        M += lpSum([yields[s][item]*var_acres_planted[item]] + [var_tons_purchased_by_scenario[s][item]] + [-1*var_tons_sold_by_scenario[s][item]]) == consumption_feed[item]
#Beans
    for item in list(set(items)-set(purchasable_items)):  #beans - constraint needs one more variable var_excess_beans_sold
        M += lpSum([yields[s][item]*var_acres_planted[item]] + [-1*var_tons_sold_by_scenario[s][item]] + [-1*var_excess_beans_sold_by_scenario[s]]) == consumption_feed[item]
        M += lpSum([var_tons_sold_by_scenario[s]["Beans"]]) <= 6000
    
                        
     

In [119]:
#Be careful - these are meant to set acres for crops and evaluate profit
# The model no longer has freedom to make decisions
"""
M += lpSum([var_acres_planted["Wheat"]]) == 100
M += lpSum([var_acres_planted["Corn"]]) == 25
M += lpSum([var_acres_planted["Beans"]]) == 375
"""

'\nM += lpSum([var_acres_planted["Wheat"]]) == 100\nM += lpSum([var_acres_planted["Corn"]]) == 25\nM += lpSum([var_acres_planted["Beans"]]) == 375\n'

In [120]:
M.writeLP("Farmer_DEP.lp")

[AVERAGE_purchase_tons_Corn,
 AVERAGE_purchase_tons_Wheat,
 AVERAGE_sell_tons_Beans,
 AVERAGE_sell_tons_Corn,
 AVERAGE_sell_tons_Wheat,
 AVERAGE_sell_tons_extra_beans,
 BAD_purchase_tons_Corn,
 BAD_purchase_tons_Wheat,
 BAD_sell_tons_Beans,
 BAD_sell_tons_Corn,
 BAD_sell_tons_Wheat,
 BAD_sell_tons_extra_beans,
 GOOD_purchase_tons_Corn,
 GOOD_purchase_tons_Wheat,
 GOOD_sell_tons_Beans,
 GOOD_sell_tons_Corn,
 GOOD_sell_tons_Wheat,
 GOOD_sell_tons_extra_beans,
 plant_acres_Beans,
 plant_acres_Corn,
 plant_acres_Wheat]

In [121]:
M.solve()
print("Status = %s" % LpStatus[M.status])
print("Profit = %f" % (M.objective.value()))

stochastic_solution_EV = M.objective.value()

Status = Optimal
Profit = 108389.999783


## Performance of this solution on the three scenarios are shown below

In [122]:
for s in scenario_keys:
    t = []
    Total_Planted = 0
    Total_Revenue = 0
    Total_Profit = 0
    t.append([s + " Yield"])
    t.append(["Item","Planted","Produced", "Purchased", "Sold","Revenue","Plant Cost","Purchase Cost","Profit"])    
    for item in items:
        purchased = 0.0
        if item in purchasable_items:
            purchased = var_tons_purchased_by_scenario[s][item].varValue

        planted = var_acres_planted[item].varValue
        planting_cost = planted* planting_costs[item]
        produced = yields[s][item] * var_acres_planted[item].varValue
        sold = var_tons_sold_by_scenario[s][item].varValue
        revenue = selling_prices[item]*sold
        if item=="Beans":
            sold += var_excess_beans_sold_by_scenario[s].varValue
            revenue += selling_price_excess_beans*var_excess_beans_sold_by_scenario[s].varValue
        purchase_cost = purchase_prices[item]*purchased
        profit = revenue - planting_cost - purchase_cost


        t.append([item, planted, produced, purchased, sold, revenue, planting_cost, purchase_cost, profit])
        Total_Planted += planted
        Total_Revenue += revenue
        Total_Profit += profit

    t.append(["","","", "", "","","","",""])    

    t.append(["Total","-",Total_Planted, "", "",Total_Revenue,"","",Total_Profit])
    t.append(["","","", "", "","","","",""])


    display_table(t)

# EVPI = Expected Value of Perfect Information
## EVPI = p(s) * (EV for Perfect decision for each scenario) - Stochastic Solution
### 1/3(59950+118600+167667) - 108390= 7016
### We can pay weather forecaster/Oracle/Machine learning expert 7k and break even if they can give us perfect information 

# VSS = Value of Stochastic Solution
## VSS = EV of DEP - EV of using Mean value solution


### Method1 : We can just take mean value solution and find optimal recourse and profit solving three scenarios and get expected value of three solution 
### Use the mean value solution and get expected profit over each scenario
### EV of mean value Solution = (55120+118600+148000)/3= 107240


### Method 2: We can also solve this by solving DEP formulation but "here and now decisions" fixed to mean  value solution



In [123]:
#This is a trick tyo compute EV of any solution by usign our DEP Formulation
M += lpSum([var_acres_planted["Wheat"]]) == 120
M += lpSum([var_acres_planted["Corn"]]) == 80
M += lpSum([var_acres_planted["Beans"]]) == 300

In [124]:
M.solve()
print("Status = %s" % LpStatus[M.status])
#print("%s = %f" % (order.name, order.varValue))
print("Profit = %f" % (M.objective.value()))
meanValue_solution_EV = M.objective.value()

Status = Optimal
Profit = 107239.999778



# VSS = 108390 - 107240 = 1150  (for p =0.3333)

In [125]:
VSS = stochastic_solution_EV - meanValue_solution_EV
print(VSS)

1150.000004350004
